# LangChain Integration - Using Tools with AI Agents

This notebook demonstrates how to integrate **llm-tool-hub** tools with **LangChain** for AI agent workflows.

## Overview

LangChain integration allows you to:
- Convert llm-tool-hub tools to LangChain StructuredTools
- Use tools with LangChain agents
- Build AI-powered automation workflows
- Chain multiple tools together

## Prerequisites

```bash
pip install langchain langchain-openai
```

## Setup

In [ ]:
import tempfile
from llm_tool_hub.filesystem_tool.create_file_tool import CreateFileTool
from llm_tool_hub.filesystem_tool.read_file_tool import ReadFileTool
from llm_tool_hub.integrations.langchain_adapter import LangchainToolAdapter

# Create a temporary directory
temp_dir = tempfile.mkdtemp()
print(f"Working directory: {temp_dir}")

# Initialize tools
create_tool = CreateFileTool(root_path=temp_dir)
read_tool = ReadFileTool(root_path=temp_dir)

print("Tools initialized")

## Example 1: Convert Tool to LangChain Format

In [ ]:
# Convert llm-tool-hub tool to LangChain StructuredTool
lc_create_tool = LangchainToolAdapter.to_langchain_structured_tool(create_tool)
lc_read_tool = LangchainToolAdapter.to_langchain_structured_tool(read_tool)

print(f"CreateFileTool converted:")
print(f"  Name: {lc_create_tool.name}")
print(f"  Description: {lc_create_tool.description}")
print(f"  Type: {type(lc_create_tool).__name__}")

print(f"\nReadFileTool converted:")
print(f"  Name: {lc_read_tool.name}")
print(f"  Description: {lc_read_tool.description}")

## Example 2: Invoke Tool Directly

In [ ]:
# Use the converted tool directly
result = lc_create_tool.invoke({
    "file_path": "test.txt",
    "content": "Hello from LangChain integration!",
    "return_content": True
})

print("Tool invocation result:")
print(result)

## Example 3: Read the Created File

In [ ]:
# Read the file we just created
result = lc_read_tool.invoke({
    "file_path": "test.txt"
})

print("File contents:")
print(result)

## Example 4: Tool Input Schema (For Agents)

In [ ]:
import json

# View the input schema that agents will use
schema = lc_create_tool.args_schema.schema()

print("CreateFileTool Input Schema:")
print(json.dumps(schema, indent=2))

## Example 5: Using with LangChain Agent (Conceptual)

This shows how you would use the tools with a LangChain agent:

In [ ]:
# NOTE: This requires OpenAI API key setup
# This is conceptual code showing how to use with agents

example_code = '''
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from llm_tool_hub.integrations.langchain_adapter import LangchainToolAdapter
from llm_tool_hub.filesystem_tool.create_file_tool import CreateFileTool
from llm_tool_hub.filesystem_tool.read_file_tool import ReadFileTool

# Initialize tools
create_tool = CreateFileTool(root_path="/workspace")
read_tool = ReadFileTool(root_path="/workspace")

# Convert to LangChain tools
tools = [
    LangchainToolAdapter.to_langchain_structured_tool(create_tool),
    LangchainToolAdapter.to_langchain_structured_tool(read_tool),
]

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run agent
result = agent.run(
    "Create a file called 'hello.py' with a simple Python function, "
    "then read it back to verify it was created correctly"
)

print(result)
'''

print("Example Agent Code:")
print("=" * 70)
print(example_code)
print("=" * 70)

## Example 6: Tool Metadata for Agents

In [ ]:
# View raw metadata (OpenAI function calling format)
metadata = create_tool.get_metadata()

print("Raw Tool Metadata (OpenAI Function Calling Format):")
print(json.dumps(metadata, indent=2))

## Key Benefits of LangChain Integration

### 1. **Structured Tool Conversion**
- Automatic schema validation with Pydantic
- Type-safe tool invocation
- Consistent interface across tools

### 2. **Agent Compatibility**
- Works with all LangChain agent types
- Supports tool calling in language models
- Automatic agent orchestration

### 3. **Error Handling**
- Automatic error recovery
- Type validation before execution
- Clear error messages for debugging

### 4. **Chaining**
- Easily chain multiple tools
- Share context between tools
- Build complex workflows

## Best Practices for Agent Workflows

1. **Use structured prompts** - Give agents clear instructions
2. **Provide context** - Include file paths and expected content
3. **Handle errors** - Agents should check tool success/failure
4. **Test workflows** - Verify agent behavior before production
5. **Monitor execution** - Use `verbose=True` for debugging

## Common Integration Patterns

### Pattern 1: File Creation and Verification
```python
# Agent creates file → Reads it back → Verifies content
```

### Pattern 2: Code Modification Workflow
```python
# Agent reads code → Analyzes it → Modifies specific sections
```

### Pattern 3: Batch Processing
```python
# Agent processes multiple files → Applies transformations
```

### Pattern 4: Shell Command Orchestration
```python
# Agent plans commands → Executes → Collects results
```

## Cleanup

## Example 7: Convert Multiple Tools at Once (List Support)

**New Feature:** You can now convert multiple tools in a single call by passing a list!

This is especially useful for setting up agents with many tools.


In [ ]:
# Method 1: Convert multiple tools individually (old way)
print("Method 1: Individual conversions")
lc_tool1 = LangchainToolAdapter.to_langchain_structured_tool(create_tool)
lc_tool2 = LangchainToolAdapter.to_langchain_structured_tool(read_tool)
tools_method1 = [lc_tool1, lc_tool2]

print(f"  Converted {len(tools_method1)} tools")

# Method 2: Convert multiple tools at once (new way - more convenient!)
print("\nMethod 2: Batch conversion with list")
tools_to_convert = [create_tool, read_tool]
lc_tools = LangchainToolAdapter.to_langchain_structured_tool(tools_to_convert)

print(f"  Converted {len(lc_tools)} tools in one call")
print(f"  Tool names: {[t.name for t in lc_tools]}")

# Method 2 is more convenient for agent setup with many tools!


In [ ]:
import shutil

# Clean up temporary directory
shutil.rmtree(temp_dir)
print(f"Cleaned up: {temp_dir}")